This notebook focuses on the embedded discrepancy strategy.

- The function MCMC_embed generates MCMC_samples for $(\lambda^1,\lambda^2)$, from "index_calib" the index of the calibration output, "idx_loo" the index of the observation $x_j$ that must be removed in the LOO scheme, "tune_size" the size of the burnin sample, "size" the sample size, and "rngseed" the random seed
- The function MCMC_multichains generate multichains MCMC samples with the same arguments as MCMC_lambda
- The function compute_error_embed takes all the GP means and standard deviations and returns the RMSRE and the levels of the predictions intervals
- The function plot_transpo takes the GP means and standard deviations and returns the prediction means and standard deviations
- The function MCMC_treat takes the index of the calibration output and the index of the observation $x_j$ that must be removed, and returns all the simulations $f(x_j, \lambda^1_k + \lambda^2_k \xi_r)$ for $1 \leq k\leq M$ and $1 \leq r\leq R$, more precisely all the GP means and standard deviations
- The function results_embed saves for each calibration index the performance metrics, and predictions means and standard deviations.


In [ ]:
def MCMC_embed(index_calib, idx_loo, tune_size, size, rngseed):
    

    def priorfun(theta, mu, sigma): #logprior function #theta = (lambda^1, lambda^2)
        lambda1 = theta[:len(index_lambda_p)]
        lambda2 = theta[len(index_lambda_p):]
        if (np.all(lambda2>=0)*int(np.all((lambda1-abs(lambda2))>=0)&np.all((lambda1+abs(lambda2))<=1))) == 0: #On veut s'assurer que lambda^1 + lambda^2 \xi reste dans les bornes
            return 10**10
        else:
            return 0
    
    def ssfun(theta, data): #loglikelihood 
        xdata = data.xdata[0]
        ydata = data.ydata[0]
        lambda1 = theta[:len(index_lambda_p)]
        lambda2 = theta[len(index_lambda_p):]
        lambda_tot = lambda1+lambda2*u #u est un echantillon de ksi
        lambda_tot = lambda_tot*(bMAXlambda-bMINlambda)+bMINlambda 
        YY, Ystd = myCODE(lambda_tot, index = [index_calib],  std_bool = True, vectorize = True, idx_loo = idx_loo) #compute gp means and std
        YY = pd.concat([pd.DataFrame(YY[ii].iloc[:, 0]) for ii in range(len(YY))], axis=1)
        Ystd = pd.concat([pd.DataFrame(Ystd[ii].iloc[:, 0]) for ii in range(len(Ystd))], axis=1)
        means = np.apply_along_axis(np.mean, 1, YY) #compute the means of the outputs
        stds = np.sqrt(np.apply_along_axis(np.var,1,YY) + np.apply_along_axis(np.mean,1, Ystd**2) + sigma[index_calib-1]**2) #compute the stds of the outputs
        ss = np.prod(norm.pdf(ydata[:,0], loc = means, scale = stds)) #independent normal approximation
        return -2*np.log(ss) 

    mcstat = MCMC(rngseed=rngseed)
    x = np.array(list(set(range(len(results_measures))) - set([idx_loo])))
    y = results_measures.loc[list(set(range(len(results_measures))) - set([idx_loo])),f"Y{index_calib}"].values
    mcstat.data.add_data_set(x, y)
    mcstat.simulation_options.define_simulation_options(
        nsimu=int(size+tune_size),
        updatesigma=False,verbosity = 0, waitbar= False)
    mcstat.model_settings.define_model_settings(sos_function=ssfun, prior_function = priorfun)

    for ii in range(len(index_lambda_p)):
        mcstat.parameters.add_model_parameter(
            name=str('$lambda1_{}$'.format(ii + 1)),
            theta0=0.5
            )
    for ii in range(len(index_lambda_p)):
        mcstat.parameters.add_model_parameter(
            name=str('$lambda2_{}$'.format(ii + 1)),
            theta0=0.2
            )

    mcstat.run_simulation()

    return mcstat.simulation_results.results


def MCMC_multichains(index_calib, idx_loo,  tune_size, size, rngseed):
    np.random.seed(rngseed)
    seeds = np.random.randint(1000, size = num_chain) #get seed for each chain
    res = [MCMC_embed(index_calib = index_calib, idx_loo = idx_loo, tune_size = tune_size, size = size, rngseed = ss) for ss in seeds] #run every MCMC chain
    samples_lambd_post = np.concatenate([res[i]["chain"][tune_size:,] for i in range(len(res))]) #concatenate the chains and remove burnin
    save_results(data = pd.DataFrame(samples_lambd_post), file = f"lambd_post_{idx_loo}.csv", pre_path = pre_path, calib=index_calib)

    

In [ ]:
def compute_error_embed(simus, stds):
    res = []
    res_intervals = []
    for idx in [1,2,3]:
        pred = np.apply_along_axis(np.mean, 0,simus[idx-1]) #get mean prediction
        error = dist2(pred,true_values[f"Y{idx}"])
        res.append(error)
        eta = abs(pred-true_values[f"Y{idx}"]).values
        intervals = np.apply_along_axis(np.mean, 0, norm.cdf((pred+eta-simus[idx-1])/stds[idx-1]) - norm.cdf((pred-eta-simus[idx-1])/stds[idx-1])) #get levels of interval prediction
        res_intervals.append(intervals)
    return res, np.transpose(np.array(res_intervals))

def plot_transpo(simus, stds):
    res_pred = pd.DataFrame()
    res_var = pd.DataFrame()
    for idx in [1,2,3]: 
        res_pred = pd.concat([res_pred,pd.DataFrame(np.apply_along_axis(np.mean, 0,simus[idx-1]))], axis=1) #get prediction mean
        res_var = pd.concat([res_var,pd.DataFrame(np.apply_along_axis(np.var, 0,simus[idx-1]) + np.apply_along_axis(np.mean, 0,stds[idx-1]**2) )], axis=1) #get prediction variance
    return res_pred, np.sqrt(res_var)

def MCMC_treat(index_calib, idx_loo):
    lambda_post = pd.read_csv(pre_path + f"/calib_{index_calib}/lambd_post_{idx_loo}.csv", index_col = 0).values #sample of (lambda^1, lambda^2
    Ysimu = pd.DataFrame(np.zeros((0,3)))
    Ystd = pd.DataFrame(np.zeros((0,3)))
    for i in range(len(lambda_post)):
        lambda1 = lambda_post[i,:len(index_lambda_p)]
        lambda2 = lambda_post[i,len(index_lambda_p):]
        lambda_tot = lambda1 + lambda2*u #u is the sample of ksi
        lambda_tot = np.apply_along_axis(lambda x: transform_Lambda(x, index_lambda_p, index_lambda_q), 1, lambda_tot) #compute GP means and stds
        res, res_std = myCODE(lambda_tot, index = [1,2,3],  std_bool = True, vectorize = True, idx_loo = idx_loo, new_x = True) #compute GP means and stds
        Ysimu = pd.concat([Ysimu, res], axis = 0) 
        Ystd = pd.concat([Ystd, res_std], axis = 0)
    return Ysimu, Ystd #returns all gp means and stds

In [ ]:
def results_embed(index_calib):
    print("A")
    results = Parallel(n_jobs=-1)(delayed(lambda idx_loo: MCMC_multichains(index_calib = index_calib, idx_loo = idx_loo, tune_size = tune_size, size = size, rngseed = rngseed))(idx_loo) for idx_loo in range(10))
    print("B")
    results_bis = Parallel(n_jobs=-1)(delayed(lambda idx_loo: MCMC_treat(index_calib = index_calib, idx_loo = idx_loo))(idx_loo) for idx_loo in range(10))
    YYtot = [pd.concat([results_bis[jj][0].iloc[:,idx_y] for jj in range(10)], axis=1) for idx_y in range(3)]
    YYstdtot = [pd.concat([results_bis[jj][1].iloc[:,idx_y] for jj in range(10)], axis=1) for idx_y in range(3)]

    errors = compute_error_embed(YYtot, YYstdtot)
    save_results(data = pd.DataFrame(errors[0]), file = "error_pred.csv", pre_path = pre_path, calib=index_calib)
    save_results(data = pd.DataFrame(errors[1]), file = "interv_errors.csv", pre_path = pre_path, calib=index_calib)

    to_plot = plot_transpo(YYtot, YYstdtot)
    save_results(data = to_plot[0], file = "predictions.csv", pre_path = pre_path, calib=index_calib)
    save_results(data = to_plot[1], file = "std_dev.csv", pre_path = pre_path, calib=index_calib)
    


In [ ]:
index_lambda_p = [0,1,2,3,4,5]
index_lambda_q = []

R = 500 #number of samples for ksi

np.random.seed(10)
lhs = qmc.LatinHypercube(d = len(index_lambda_p), scramble=False, optimization="random-cd", seed = 123) #sample for ksi is obtained with LHS as ksi is uniform
u = lhs.random(n=R)*2 - 1

tune_size = 3000
size = 2000
rngseed = 432
num_chain = 1

[results_embed(index_calib) for index_calib in calib_only]